In [98]:
import random
import pandas as pd
import sys
from tabulate import tabulate

# Chef Tournament 
* Four divisions
* Eight chefs in each division
* Each chef is ranked in each division
* In the first round of matches, chefs are paired by seed ranking,
  i.e. Seed 1/Seed 8, Seed 2/Seed 7, Seed 3/Seed 6, Seed 4/Seed 5

## Functions

In [99]:
def isPowerOfTwo (n):
    """ Check number is a power of Two 
    Parameter:
    n: integer
    
    Returns:
    Boolean: Returning Value
    """
    
    # Stackoverflow
    return (n & (n-1) == 0) and n != 0

In [100]:
def IsLengthEight(df_length):
    """ Is dataframe length or number equal to 8?

    Parameter:
    df_length: integer

    Returns:
    Boolean: Returning Value
    """
    
    if df_length != 8:
        return False
    return True
    

In [101]:
def chefmatch(chef1, chef2, printresults = True):
    """ Simulates a match between two chefs and randomly picks a winner

      Parameters:
      chef1 (string):  Name of first Chef 
      chef2 (string):  Name of second Chef 
      printresults (boolean): print the results of the match

      Returns:
      winner(string): Name of chef selected as the winner

     """"
    # randomly choose a winner
    winner = random.choice([chef1, chef2])
    if printresults:
        #print(f' {chef1} vs {chef2} ==> Winner: {winner}')
        print('{:<25} {:<3} {:<25} {:<12} {:<25}'.format(chef1,"vs", chef2,"==> Winner: ", winner))
    return winner

In [102]:
def roundOneBySeed(df, team):
    """ Single elimination match.  Chefs are paired by seed.  Winner 
        moves to the next round.  Each team is processed separately.

    Parameters:
    df (dataframe): dataframe with teams and chefs
    team (Series): A team of chefs

    Returns:
    winners (list): Names of the winners of each match

    """
    
    no_players = len(df)  # length of dataframe
    no_matches = int(no_players/2)  # number of matches: length of df /2
    winners = []
    print("\n", team, "Round 1:")
    for i in range(0,no_matches):
        
        chef1 = df.iloc[i][team]
      
        chef2 = df.iloc[no_players-i-1][team]
        winner = chefmatch(chef1,chef2,printresults=False)
       # print(f'seed {i}: {chef1} , seed {no_players-i}: {chef2} ==> Winner: {winner}')
        print('{:<3} {:<25} {:<3} {:<3} {:<25} {:<12} {:<25}'.format(i+1,chef1,"vs", no_players-i,chef2,"==> Winner: ", winner))
        winners.append(winner)  # append name of winning chef to list
    return winners

In [103]:
def tournamentround(winners):
    """ The Winners of the preliminary seed matches and paired together for the next
         round of matches.  

         Parameters:
         winners (list): Names of the winners from the preliminary seed matches

         Returns:
         next_round_winners (list): Names of the winners of the matches
    """
    
    next_round_winners = []
              
    for i in range(0,len(winners),2):
           
        chef1 = winners[i]
        chef2 = winners[i+1] if i+1 < len(winners) else None
      
        # chef match
        winner = chefmatch(chef1, chef2, printresults=True)
        # Add winner to winners list
        next_round_winners.append(winner)
   
    return next_round_winners

In [104]:
def semifinals(first_round_winners, team):
    """ Winning chefs from the tournament round are paired for matches.  The Winners will move to the 
        finals.  Each team will send one person to the finals.

        Parameters:
        first_round_winners (list): List of winners
        team (string): name of the team

        Returns:
        semifinal_winner (list): Semifinal winner

    """
    # check the number of winners is divisible by 2
    if len(first_round_winners) % 2 !=0:
        return ['Number of players must be divisible by 2']

    # Make a copy of the winners list
    winners = first_round_winners[:] 

    round_number = 1 # initialize round number to 1
    
    # Iterate over matches between winners
    while len(winners) > 1:
        print("\n", team, "Semi-finals Round: ", round_number)
        
        winners = tournamentround(winners)
           
        round_number +=1
    semifinal_winner = winners[0]
    print("\n", "The ", team, "winner is: ", semifinal_winner)
    return semifinal_winner

In [105]:
def finals(semifinals_winners):
    # check the number of winners is divisible by 2
    if len(semifinals_winners) % 2 !=0:
        return ['Number of players must be divisible by 2']

    # Make a copy of the winners list
    winners = semifinals_winners[:] 

    round_number = 1 # initialize round number to 1
    
    # Iterate over matches between winners
    while len(winners) > 1:
        print("\n", "Finals Round: ", round_number)
        
        winners = tournamentround(winners)
           
        round_number +=1
    
    champ = winners[0]
    return champ

In [106]:
def tournament(df):
    region_champs = []
    
    for colm in df.columns:
    
        roundone_winners = roundOneBySeed(df,colm)
        semifinal_winner = semifinals(roundone_winners, colm)
        region_champs.append(semifinal_winner)
        roundone_winners = []
        semifinal_winner = []
   
    champion = finals(region_champs)
   
    return champion

## Meet the Chefs

In [107]:
def main():
    # meet the chefs

    df = pd.read_csv("../Python/Tournamentchefs.csv")
    
    if  not (isPowerOfTwo(df.shape[1]) and IsLengthEight(df.shape[0])):
       print("Exiting program.  Requires 8 chefs per team. Number of teams must be a power of 2.")
       sys.exit(1)

    # Print chef groups
    print(tabulate(df, headers='keys', tablefmt='psql',showindex=False))

    # Run the tournament
    tournament_champion = tournament(df)
    print("\n" "The Tournament Champion is: ", tournament_champion)

## Tournament

In [108]:
if __name__ == "__main__":
   
    main()

+----------------------+-------------------+----------------------+-------------------+
| West Coast A         | West Coast B      | East Coast A         | East Coast B      |
|----------------------+-------------------+----------------------+-------------------|
| Jet Tila             | Antonia Lofaso    | Tiffani Faison       | Tobias Dorzon     |
| Shirley Chung        | Tiffany Derry     | Jose Garces          | Maneet Chauhan    |
| Nate Appleman        | Mei Lin           | Graham Elliot        | Stephanie Izard   |
| Brian Malarkey       | Joe Sasto         | Rocco Dispirito      | Karen Akunowicz   |
| Lee Anne Wong        | Elizabeth Falkner | Ilan Hall            | Christian Petroni |
| Adam Sobel           | Crista Luedtke    | Eric Adjepong        | Madison Cowan     |
| Tracey Shepos Cenami | Viet Pham         | Kelsey Barnard Clark | Jonathon Sawyer   |
| Carlos Anthony       | Shota Nakajima    | Britt Rescigno       | Leah Cohen        |
+----------------------+--------